In [ ]:
import os
import re # Import library regular expression
import string # Import string untuk daftar tanda baca standar
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import time

# --- Konfigurasi ---
NAMA_FOLDER_INPUT = 'input/indo'
NAMA_FOLDER_OUTPUT_RELATIF = 'output/sastrawi'
# --- Akhir Konfigurasi ---

# Dapatkan path direktori kerja saat ini (cocok untuk notebook/interactive)
base_dir = os.getcwd()
print(f"Menggunakan direktori kerja saat ini sebagai basis: {base_dir}")
input_folder_path = os.path.join(base_dir, NAMA_FOLDER_INPUT)
output_folder_path = os.path.join(base_dir, NAMA_FOLDER_OUTPUT_RELATIF)

# --- Fungsi untuk Membersihkan Teks ---
def bersihkan_teks(teks):
    """Membersihkan teks dari tanda baca, angka, dan spasi berlebih."""
    # 1. Ubah ke lowercase
    teks = teks.lower()
    # 2. Hapus angka
    teks = re.sub(r"\d+", "", teks)
    # 3. Hapus tanda baca
    # Membuat pola regex dari semua tanda baca di string.punctuation
    tanda_baca_escaped = re.escape(string.punctuation)
    teks = re.sub(r'[' + tanda_baca_escaped + ']', '', teks)
    # 4. Hapus spasi putih berlebih (termasuk newline, tab) dan strip spasi di awal/akhir
    teks = re.sub(r'\s+', ' ', teks).strip()
    return teks
# --- Akhir Fungsi Cleaning ---

# 1. Inisialisasi Stemmer Sastrawi
print("Menginisialisasi stemmer Sastrawi...")
factory = StemmerFactory()
stemmer = factory.create_stemmer()
print("Stemmer siap.")

# 2. Buat folder output jika belum ada
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    print(f"Folder output dibuat: {output_folder_path}")
else:
    print(f"Folder output sudah ada: {output_folder_path}")

# 3. Proses setiap file dalam folder input
print(f"\nMemulai proses cleaning dan stemming dari folder: {input_folder_path}")
start_time = time.time()

try:
    if not os.path.isdir(input_folder_path):
        raise FileNotFoundError(f"Folder input '{input_folder_path}' tidak ditemukan atau bukan direktori.")

    list_file = os.listdir(input_folder_path)
    processed_files = 0
    skipped_files = 0

    if not list_file:
        print("Folder input kosong, tidak ada file yang diproses.")

    for filename in list_file:
        input_file_path = os.path.join(input_folder_path, filename)

        if os.path.isfile(input_file_path) and filename.lower().endswith('.txt'):
            output_file_path = os.path.join(output_folder_path, filename)

            print(f"  -> Memproses file: {filename}...")

            try:
                # Baca konten file input
                with open(input_file_path, 'r', encoding='utf-8', errors='ignore') as f_in:
                    original_text = f_in.read()

                # *** Lakukan Cleaning Teks *** <--- Langkah Baru
                cleaned_text = bersihkan_teks(original_text)

                # Lakukan stemming pada teks yang sudah dibersihkan
                stemmed_text = stemmer.stem(cleaned_text) # Gunakan cleaned_text

                # Tulis hasil stemming ke file output
                with open(output_file_path, 'w', encoding='utf-8') as f_out:
                    f_out.write(stemmed_text)

                processed_files += 1

            except Exception as e:
                print(f"     ERROR saat memproses file {filename}: {e}")
        else:
            if os.path.isfile(input_file_path):
                 print(f"  -> Melewati file non-txt: {filename}")
            else:
                 print(f"  -> Melewati item yang bukan file: {filename}")
            skipped_files += 1

    end_time = time.time()
    total_time = end_time - start_time

    print(f"\nProses cleaning dan stemming selesai.")
    print(f"Jumlah file .txt yang diproses: {processed_files}")
    if skipped_files > 0:
        print(f"Jumlah item non-txt atau subfolder yang dilewati: {skipped_files}")
    print(f"Hasil disimpan di folder: {output_folder_path}")
    print(f"Total waktu eksekusi: {total_time:.2f} detik")

except FileNotFoundError as fnf_error:
    print(f"ERROR: {fnf_error}")
    print("Pastikan nama folder input sudah benar dan folder tersebut ada di lokasi yang sama dengan skrip/notebook.")
except Exception as e:
    print(f"Terjadi kesalahan umum yang tidak terduga: {e}")

Menggunakan direktori kerja saat ini sebagai basis: /home/xerces/project/stemming-project
Menginisialisasi stemmer Sastrawi...
Stemmer siap.
Folder output sudah ada: /home/xerces/project/stemming-project/output/sastrawi

Memulai proses stemming dari folder: /home/xerces/project/stemming-project/input
  -> Memproses file: Kierkegaard_ A Brief Introduction_1.txt...

Proses stemming selesai.
Jumlah file .txt yang diproses: 1
Hasil disimpan di folder: /home/xerces/project/stemming-project/output/sastrawi
Total waktu eksekusi: 13.40 detik
